<a href="https://colab.research.google.com/github/happycindy7201/chatGPT-NLP/blob/main/mooc4_2_4%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B82_RNN_LSTM_%E1%84%80%E1%85%A1%E1%86%B7%E1%84%89%E1%85%A5%E1%86%BC%E1%84%87%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5%E1%86%A8_%E1%84%89%E1%85%AE%E1%84%80%E1%85%A1%E1%86%BC%E1%84%89%E1%85%A2%E1%86%BC%E1%84%8B%E1%85%AD%E1%86%BC_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 정의

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 문장의 감정 (긍정, 부정) 분류

# 데이터 세트

- 데이터 세트명 : 한국어 네이버 영화 리뷰
- 데이터 세트 출처 : https://github.com/e9t/nsmc

# 데이터 세트 상세 설명

- 각 파일은 id, document, label 세 개의 열로 구성됩니다.
- 열은 탭으로 구분됩니다(예: .tsv형식. 그러나 파일 확장자는 .txt초보자가 쉽게 접근할 수 있도록 함).
- 총 리뷰 20만개
  - ratings.txt: 전체 리뷰 20만개
  - ratings_test.txt: 테스트용 리뷰 50,000개
  - ratings_train.txt: 훈련용 리뷰 150,000개

- id: 리뷰 아이디, 네이버에서 제공
- document: 실제 리뷰
- label: 리뷰의 감정 클래스입니다. (0: 부정, 1: 긍정)


In [1]:
# 한글 형태소 분석 패키지 설치
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 6.3 MB/s eta 0:00:00
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.19.0-py3-none-any.whl size=34793896 sha256=659e5a1821e2d45f70e61a9f00cfc8c76e0fdf65f0832aca76d8d89e7389ee2c
  Stored in directory: /root/.cache/pip/wheels/7a/1b/f4/91a56e5f2e34c0f981058fca3dca6bb7f02ff20d69d8b53c3b
Successfully built kiwipiepy-model


In [2]:
# 패키지 로드
import numpy as np
import pandas as pd
import tensorflow as tf

# 한글 형태소 분석 패키지 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()


In [3]:
# 훈련 데이터 불러오기 (긍정 1, 부정 0)
train_data = pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
train_data = train_data[:1000]
train_data.shape, display(train_data.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


((1000, 3), None)

In [4]:
# 검증 데이터 불러오기
test_data = pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
test_data = test_data[:100]
test_data.shape, display(test_data.head())

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


((100, 3), None)

In [5]:
# 주요 전처리 (.....생략......)

In [6]:
# document 토큰화 전처리 - 1 (형태소 분석기 확인)
kiwi.tokenize('한글이 좋아합니다.')

[Token(form='한글', tag='NNG', start=0, len=2),
 Token(form='이', tag='JKS', start=2, len=1),
 Token(form='좋', tag='VA', start=4, len=1),
 Token(form='어', tag='EC', start=5, len=1),
 Token(form='하', tag='VX', start=6, len=1),
 Token(form='ᆸ니다', tag='EF', start=6, len=3),
 Token(form='.', tag='SF', start=9, len=1)]

In [7]:
kiwi.tokenize('한글을 사랑합니다.')

[Token(form='한글', tag='NNG', start=0, len=2),
 Token(form='을', tag='JKO', start=2, len=1),
 Token(form='사랑', tag='NNG', start=4, len=2),
 Token(form='하', tag='XSV', start=6, len=1),
 Token(form='ᆸ니다', tag='EF', start=6, len=3),
 Token(form='.', tag='SF', start=9, len=1)]

In [8]:
# document 토큰화 전처리 - 2 (훈련 / 검증 데이터 토큰화)
x_train = []
for x in train_data['document']:
    tokens = [token[0] for token in kiwi.tokenize(x)] # 토큰화
    x_train.append(tokens)

x_test = []
for x in test_data['document']:
    tokens = [token[0] for token in kiwi.tokenize(x)] # 토큰화
    x_test.append(tokens)

In [9]:
# document 토큰화 전처리 - 3 (정수 인코딩)
# 훈련 데이터를 기준으로 각 토큰에 대해 정수 인코딩
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(x_train)

# 각 토큰 정수 값을 이용하여 모든 토큰을 숫자로 변경
x_train_ = tokenizer.texts_to_sequences(x_train)
x_test_ = tokenizer.texts_to_sequences(x_test)

# 50개 토큰으로 제한하여 줄이거나, 자르기
x_train_ = tf.keras.preprocessing.sequence.pad_sequences(x_train_, maxlen=50)
x_test_ = tf.keras.preprocessing.sequence.pad_sequences(x_test_, maxlen=50)

In [10]:
# 정답값 준비
y_train_ = train_data['label']
y_test_ = test_data['label']

In [11]:
x_train_


array([[   0,    0,    0, ...,   22,   58,  840],
       [   0,    0,    0, ...,   23,   70,  491],
       [   0,    0,    0, ...,  364,    2,   49],
       ...,
       [   0,    0,    0, ...,  291,  764,   36],
       [   0,    0,    0, ...,   59,   49,    4],
       [   2,  283,   10, ...,   47, 3209,    4]], dtype=int32)

In [12]:
# LSTM 모델 입력 데이터로 차원변환
x_train_ = x_train_.reshape(-1, 1, 50)
x_test_ = x_test_.reshape(-1, 1, 50)

# 모델링

In [40]:
# 모델 정의
i = tf.keras.Input(shape=(None, 50, ))
#차원축소를 막이 위해
x = tf.keras.layers.LSTM(64, activation='relu', return_sequences=True)(i)
x = tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)(x)
x = tf.keras.layers.LSTM(16, activation='relu', return_sequences=True)(x)
#x = tf.keras.layers.LSTM(8, activation='relu', return_sequences=True)(x)
x = tf.keras.layers.LSTM(4, activation='relu', return_sequences=True)(x)
x = tf.keras.layers.LSTM(2, activation='relu', return_sequences=True)(x)
o = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=i, outputs=o, name='no3_model')

# 모델 학습 과정 설정
#loss부분은 수정하면 안됨
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
hist = model.fit(x_train_, y_train_, epochs=100, batch_size=64)


Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 820ms/step - accuracy: 0.4841 - loss: 0.6968
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5202 - loss: 0.6937 
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5893 - loss: 0.6907 
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6231 - loss: 0.6880 
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6280 - loss: 0.6870 
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6199 - loss: 0.6814 
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6021 - loss: 0.6727 
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6075 - loss: 0.6718 
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6125 - loss: 0.6675 
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5990 - loss: 0.6631 
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6021 - loss: 0.6550 
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [41]:
model.summary()

Model: "no3_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, None, 50)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_38 (LSTM)                       │ (None, None, 64)            │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_39 (LSTM)                       │ (None, None, 32)            │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_40 (LSTM)                       │ (None, None, 16)            │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_41 (LSTM)                       │ (None, None, 4)             │             336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_42 (LSTM)                       │ (None, None, 2)             │              56 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, None, 1)             │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 136,163 (531.89 KB)

 Trainable params: 45,387 (177.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 90,776 (354.60 KB)

# 모델 평가

In [42]:
# 모델 평가
score = model.evaluate(x_test_, y_test_)
print("%s: %.2f%%" %(model.metrics_names[1], score[1]*100))


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 670ms/step - accuracy: 0.6035 - loss: 0.7367
compile_metrics: 60.00%


In [43]:
# score 점수를 파일로 저장
fname = 'Week4_2.txt'
f = open(f'{fname}', 'w')
f.write(str(score[1]))
f.close()

# 예측

In [44]:
# 예측
pred = model.predict(x_test_)
pred = pred.reshape(-1)
test_data['pred'] = np.where(pred >= 0.5, 1, 0)
test_data

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 431ms/step


,id,document,label,pred
0,6270596,굳 ㅋ,1,0
1,9274899,GDNTOPCLASSINTHECLUB,0,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,1
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,0
...,...,...,...,...
95,6753658,전기톱은못들고다니는데 엔진톱이겠죠,0,0
96,9665771,완전 재밌엇는데 왜 평점이??,1,0
97,8757576,제임스 완이 내 목표임 ㄷ,1,1
98,9850643,1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴...,0,1


# evaluate() 이해

In [45]:
# model.evaluate(x_test_, y_test_) 이해
np.sum(test_data['label'] == test_data['pred']) / 100 * 100

60.0

In [ ]:
# 패키지 로드
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 한글 형태소 분석 패키지 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()

# 훈련 데이터 불러오기 (긍정 1, 부정 0)
train_data = pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
train_data = train_data[:5000]
train_data.shape, display(train_data.head())

# 검증 데이터 불러오기
test_data = pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
test_data = test_data[:100]
test_data.shape, display(test_data.head())

# document 토큰화 전처리 - 1 (형태소 분석기 확인)
kiwi.tokenize('한글을 사할합니다.')

# document 토큰화 전처리 - 2 (훈련 / 검증 데이터 토큰화)
x_train = []
for x in train_data['document']:
    tokens = [token[0] for token in kiwi.tokenize(x)] # 토큰화
    x_train.append(tokens)

x_test = []
for x in test_data['document']:
    tokens = [token[0] for token in kiwi.tokenize(x)] # 토큰화
    x_test.append(tokens)

# document 토큰화 전처리 - 3 (정수 인코딩)
from collections import Counter
from itertools import chain

# Flatten the list of token lists and count word frequencies
all_tokens = list(chain(*x_train))
word_freq = Counter(all_tokens)

# Create a mapping from words to indices
vocab = {word: i+1 for i, (word, _) in enumerate(word_freq.most_common())}

# Encode the documents
x_train_ = [[vocab[word] for word in doc if word in vocab] for doc in x_train]
x_test_ = [[vocab[word] for word in doc if word in vocab] for doc in x_test]

# Pad sequences
def pad_sequences(sequences, maxlen):
    padded = np.zeros((len(sequences), maxlen))
    for i, seq in enumerate(sequences):
        if len(seq) > 0:
            padded[i, -len(seq):] = np.array(seq)[:maxlen]
    return padded

x_train_ = pad_sequences(x_train_, maxlen=100)
x_test_ = pad_sequences(x_test_, maxlen=100)

# 정답값 준비
y_train_ = train_data['label'].values
y_test_ = test_data['label'].values

# LSTM 모델 입력 데이터로 차원변환
x_train_ = x_train_.reshape(-1, 1, 100)
x_test_ = x_test_.reshape(-1, 1, 100)

# Create PyTorch datasets and loaders
train_dataset = TensorDataset(torch.tensor(x_train_, dtype=torch.float32), torch.tensor(y_train_, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(x_test_, dtype=torch.float32), torch.tensor(y_test_, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        out = self.sigmoid(out)
        return out

model = LSTMModel(input_size=100, hidden_size=64, output_size=1)

# 모델 학습 과정 설정
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 50
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() >= 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total * 100
print(f'Accuracy: {accuracy:.2f}%')

# score 점수를 파일로 저장
fname = 'Week4_2.txt'
with open(fname, 'w') as f:
    f.write(str(accuracy))

# 예측
test_preds = []
model.eval()
with torch.no_grad():
    for inputs, _ in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() >= 0.5).float()
        test_preds.extend(predicted.numpy())

test_data['pred'] = test_preds
print(test_data)

# model.evaluate equivalent
evaluation_accuracy = np.sum(test_data['label'] == test_data['pred']) / len(test_data) * 100
print(f'Evaluation Accuracy: {evaluation_accuracy:.2f}%')

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


Accuracy: 49.00%
         id                                           document  label  pred
0   6270596                                                굳 ㅋ      1   0.0
1   9274899                               GDNTOPCLASSINTHECLUB      0   1.0
2   8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0   0.0
3   6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0   1.0
4   6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0   1.0
..      ...                                                ...    ...   ...
95  6753658                                 전기톱은못들고다니는데 엔진톱이겠죠      0   1.0
96  9665771                                   완전 재밌엇는데 왜 평점이??      1   1.0
97  8757576                                     제임스 완이 내 목표임 ㄷ      1   1.0
98  9850643  1점고 아깝다. 개막장 영화의 원조라고나 할까.아내와 사별한 지 얼마나 지났다고 딴...      0   0.0
99  9985753  가끔 문득 생각나서 다시보는 영화..색감이 정말 예술이죠 강렬하고 화려하고..츠지야...      1   1.0

[100 rows x 4 columns]
Evaluation Accuracy: 49.00%


# [실습 평가 방법]

본 프로그램을 실행한 후 생성되는 y_pred.csv 파일을 실습 평가 사이트에 등록하여 0.55 이상 획득 필요

In [46]:
# 모델 추론
y_pred_prob = model.predict(x_test_)
# model.eval()  # Set the model to evaluation mode
#with torch.no_grad():  # Disable gradient calculations
    #y_pred_prob = model(torch.tensor(x_test_, dtype=torch.float32))  # Pass the test data through the mode

# 확률을 클래스 라벨로 변환 (0.5를 기준으로)
# y_pred = (y_pred_prob > 0.5).astype(int) # Original line causing the error
y_pred = (y_pred_prob > 0.5).astype(int).squeeze() # Squeeze to remove extra dimension

# y_pred를 DataFrame으로 변환 후 CSV 파일로 저장
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.to_csv('y_pred_4_2_12400662.csv', index=False, header=False)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


<font size=5> [예측결과 제출](https://aifactory.space/task/2594/overview) </font>

# End Of Program